```
███████  █████  ███████ ████████ ████████ ██████   █████  ███    ██ ███████ ███████ ███████ ██████  
██      ██   ██ ██         ██       ██    ██   ██ ██   ██ ████   ██ ██      ██      ██      ██   ██ 
█████   ███████ ███████    ██       ██    ██████  ███████ ██ ██  ██ ███████ █████   █████   ██████  
██      ██   ██      ██    ██       ██    ██   ██ ██   ██ ██  ██ ██      ██ ██      ██      ██   ██ 
██      ██   ██ ███████    ██       ██    ██   ██ ██   ██ ██   ████ ███████ ██      ███████ ██   ██ 

# Demo 3 - Source PGSQL16 - Target PGSQL17

In [1]:
#r "nuget:Microsoft.DotNet.Interactive.PostgreSQL , *-*"

Installed Packages Microsoft.DotNet.Interactive.PostgreSQL, 1.0.0-beta.25323.1

Loading extension script from `C:\Users\romai\.nuget\packages\microsoft.dotnet.interactive.postgresql\1.0.0-beta.25323.1\interactive-extensions\dotnet\extension.dib`

Query PostgreSQL databases. 
 This extension adds support for connecting to PostgreSql databases using the #!connect postgres magic command.

## Working on TPCH SF10 ORDERS (15M rows)

### PGSQL 16 to PG 17

In [2]:
// Connect to PostgreSQL target database
#!connect postgres "Host=localhost;Port=25432;Database=tpch;Username=FastUser;Password=FastPassword;CommandTimeout=1500" --kernel-name pgsql17

Kernel added: #!sql-pgsql17

## Copy data from postgresql 16 to postgresql 17 using dblink (to compare)

#### Target orders_3 with a clustered ColumnStoreIndex

In [3]:
TRUNCATE TABLE tpch_10.orders;
SELECT count(*) count_target_table_rows from tpch_10.orders;

ALTER TABLE tpch_10.orders SET UNLOGGED;

count_target_table_rows
0


In [4]:
DROP FOREIGN TABLE IF EXISTS "pg16_orders";
DROP USER MAPPING if exists for "FastUser" server "pg16tpch";
DROP SERVER if exists "pg16tpch";

In [5]:
CREATE SERVER "pg16tpch" FOREIGN DATA WRAPPER postgres_fdw OPTIONS (host 'localhost', dbname 'tpch', port '15432');
CREATE USER MAPPING IF NOT EXISTS FOR "FastUser" SERVER pg16tpch OPTIONS (user 'FastUser', password 'FastPassword');

In [6]:
-- Create external table in pg17 (target) to read data from pg16 (source)
DROP FOREIGN TABLE IF EXISTS pg16_orders;
CREATE FOREIGN TABLE IF NOT EXISTS pg16_orders (
    o_orderkey integer,
    o_custkey integer,
    o_orderstatus char(1),
    o_totalprice numeric(15,2),
    o_orderdate date,
    o_orderpriority char(15),
    o_clerk char(15),
    o_shippriority integer,
    o_comment varchar(79)
) SERVER pg16tpch OPTIONS (schema_name 'tpch_10', table_name 'orders');

### Copy data from postgresql 16 to postgresql 17 using the external table ==>


In [7]:
INSERT INTO tpch_10.orders
SELECT * FROM pg16_orders;

In [8]:
SELECT count(*) count_target_table_rows_after_insert from tpch_10.orders;

count_target_table_rows_after_insert
15000000


## Test FastTransfer with Method Ctid (postgresql only)

CTID is a system hidden column that stores the physical location (block number and tuple offset) of a row version within its table.

You can SELECT it and even filter data using this hidden column. FastTransfer use the Ctid column to select data by chunck. It also create a micro-postgresql-snapshot before openning several connections to the source in order to have a stable source for each threads that will extract a portion of the data. 

In [3]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "pgcopy" `
--sourceserver "localhost:15432" `
--sourcedatabase "tpch" `
--sourceuser "FastUser" `
--sourcepassword "FastPassword" `
--sourceschema "tpch_10" `
--sourcetable "orders" `
`
--targetconnectiontype "pgcopy" `
--targetserver "localhost:25432" `
--targetuser "FastUser" `
--targetpassword "FastPassword" `
--targetdatabase "tpch" `
--targetschema "tpch_10" `
--targettable "orders" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ctid" `
--degree -3 `
--runid "PG16_to_PG17"

2025-10-03T11:40:36.298 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- FastTransfer - running in trial mode – will expires on 2025‑10‑27 (23 day(s) left).
2025-10-03T11:40:36.306 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- Starting
2025-10-03T11:40:36.307 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- FastTransfer Version : 0.13.12.0 Architecture : X64 - Framework : .NET 8.0.20
2025-10-03T11:40:36.307 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- OS : Microsoft Windows 10.0.26100
2025-10-03T11:40:36.307 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- Process ID : 95872
2025-10-03T11:40:36.307 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- INFORMATION -|- tpch.tpch_10.orders -|- Provided Run ID : PG16_to_PG17
2025-10-03T11:40:36.307 +02:00 -|- FastTransfer -|- PG16_to_PG17 -|- INFORMATION -|- tpch.t

In [10]:
SELECT count(*) count_target_table_rows from tpch_10.orders;

count_target_table_rows
15000000


In [11]:
SELECT * FROM tpch_10.orders LIMIT 5;

o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
23800896,1245896,F,57080.000,1994-10-03 00:00:00Z,2-HIGH,Clerk#000005799,0,gifts. special accounts grow blithely al
23800897,1257877,O,136200.000,1997-02-20 00:00:00Z,3-MEDIUM,Clerk#000004630,0,quickly final pack
23800898,128710,O,201374.000,1997-05-20 00:00:00Z,5-LOW,Clerk#000000178,0,lites haggle blithely unusual accounts. sly
23800899,228931,F,153592.000,1993-08-01 00:00:00Z,3-MEDIUM,Clerk#000008291,0,ckages nod even instructions. fluffily ironic accounts cajole above the
23800900,114088,O,72678.000,1996-06-20 00:00:00Z,5-LOW,Clerk#000005197,0,pearls wake final theodolites. unusual instructions sleep blithely.


#### Target orders_heap : Heap Table

In [12]:
DROP TABLE IF EXISTS tpch_10.orders_heap;

CREATE TABLE tpch_10.orders_heap AS
SELECT * 
FROM tpch_10.orders
WHERE 1=0;

ALTER TABLE tpch_10.orders_heap SET UNLOGGED;

SELECT count(*) count_orders_heap from tpch_10.orders_heap;

count_orders_heap
0


In [ ]:
TRUNCATE TABLE tpch_10.orders_heap;
INSERT INTO tpch_10.orders_heap
SELECT * FROM pg16_orders;


In [2]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "pgcopy" `
--sourceserver "localhost:15432" `
--sourcedatabase "tpch" `
--sourceuser "FastUser" `
--sourcepassword "FastPassword" `
--sourceschema "tpch_10" `
--sourcetable "orders" `
`
--targetconnectiontype "pgcopy" `
--targetserver "localhost:25432" `
--targetuser "FastUser" `
--targetpassword "FastPassword" `
--targetdatabase "tpch" `
--targetschema "tpch_10" `
--targettable "orders_heap" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ctid" `
--degree 12 `
--runid "PG16_to_PG17_heap"

2025-10-03T11:39:59.696 +02:00 -|- FastTransfer -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- FastTransfer - running in trial mode – will expires on 2025‑10‑27 (23 day(s) left).
2025-10-03T11:39:59.705 +02:00 -|- FastTransfer -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- Starting
2025-10-03T11:39:59.705 +02:00 -|- FastTransfer -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- FastTransfer Version : 0.13.12.0 Architecture : X64 - Framework : .NET 8.0.20
2025-10-03T11:39:59.705 +02:00 -|- FastTransfer -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- OS : Microsoft Windows 10.0.26100
2025-10-03T11:39:59.705 +02:00 -|- FastTransfer -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- Process ID : 67680
2025-10-03T11:39:59.706 +02:00 -|- FastTransfer -|- PG16_to_PG17_heap -|- INFORMATION -|- tpch.tpch_10.orders_heap -|- Provided Run ID : PG16_to_PG17_heap
2025-10-03T11:39:59.706 +0

In [14]:

SELECT count(*) count_rows_in_heap from tpch_10.orders_heap;

count_rows_in_heap
15000000


In [15]:
SELECT * from tpch_10.orders_heap LIMIT 5;

o_orderkey,o_custkey,o_orderstatus,o_totalprice,o_orderdate,o_orderpriority,o_clerk,o_shippriority,o_comment
42469217,1461151,O,298887.000,1998-06-06 00:00:00Z,3-MEDIUM,Clerk#000007344,0,ly express dependencies boo
42469218,1037713,O,191035.000,1997-07-23 00:00:00Z,3-MEDIUM,Clerk#000001497,0,never unusual packages sleep slyly. furiously even deposits sleep qu
42469219,339392,F,82942.000,1993-06-15 00:00:00Z,5-LOW,Clerk#000006633,0,carefully even accounts serve car
42469220,1149826,O,105800.000,1997-02-06 00:00:00Z,1-URGENT,Clerk#000007476,0,braids integrate carefully express accounts. accounts among the slyly e
42469221,595162,O,141853.000,1996-09-01 00:00:00Z,3-MEDIUM,Clerk#000003886,0,c packages. blithely regular requests boost about the ironic dolp


#### Test using a bigger table : LineItem (60M Rows x 16 columns - 16GB) 

In [5]:
cd D:\FastTransfer

.\FastTransfer.exe `
--sourceconnectiontype "pgcopy" `
--sourceserver "localhost:15432" `
--sourcedatabase "tpch" `
--sourceuser "FastUser" `
--sourcepassword "FastPassword" `
--sourceschema "tpch_10" `
--sourcetable "lineitem" `
`
--targetconnectiontype "pgcopy" `
--targetserver "localhost:25432" `
--targetuser "FastUser" `
--targetpassword "FastPassword" `
--targetdatabase "tpch" `
--targetschema "tpch_10" `
--targettable "lineitem_heap_unlogged" `
`
--loadmode "Truncate" `
--mapmethod "Name" `
--method "Ctid" `
--degree 12 `
--runid "PG16_to_PG17_60M_rowsx16_columns"

2025-10-03T11:42:22.443 +02:00 -|- FastTransfer -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- FastTransfer - running in trial mode – will expires on 2025‑10‑27 (23 day(s) left).
2025-10-03T11:42:22.452 +02:00 -|- FastTransfer -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- Starting
2025-10-03T11:42:22.452 +02:00 -|- FastTransfer -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- FastTransfer Version : 0.13.12.0 Architecture : X64 - Framework : .NET 8.0.20
2025-10-03T11:42:22.452 +02:00 -|- FastTransfer -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- OS : Microsoft Windows 10.0.26100
2025-10-03T11:42:22.452 +02:00 -|- FastTransfer -|- PG16_to_PG17_60M_rowsx16_columns -|- INFORMATION -|- tpch.tpch_10.lineitem_heap_unlogged -|- Process ID : 65196
2025-10-03T11:42:22.452 +02:00 -|- FastTransfer -|-

#### Results with Ctid pgcopy ==> pgcopy (binary copy method) :

| PG16 ==> PG17 (Ctid method) 				| Elapsed Time (seconds)|Cells/sec 	|Parallel Dispatch Method|degree of parallelism
|-----------------------------------------------------------------------|:---------------------:|----------:| :--: | :--: |
| FastTransfer Load LINEITEM (60M Rows x 16c columns) HEAP Target       | 16.1     			|60_000_000    | Ctid | 12 |
| FastTransfer Load ORDERS   (15M Rows x  9c columns) Target with PK 		| 5.7     				|23_684_210    | Ctid | 10 (-3) |
| FastTransfer Load ORDERS   (15M Rows x  9c columns) HEAP Target 		| 3.3     				|40_909_090    | Ctid | 12 |


The speed of loads (in cells/sec) is higher with LINEITEM than with ORDERS because the LINEITEM table is bigger and the startup and preparation time take a smaller role/space in the total time than it take with ORDERS load.

#### Results with foreign table + insert (legacy method) :
| PG16 ==> PG17 (Foreign Table + Insert method) 	| Elapsed Time (seconds)|Cells/sec 	|Parallel Dispatch Method
|-----------------------------------------------------------------------|:---------------------:|----------:| :--: |
| Postgresql fdw table  (15M Rows x  9c columns) Target with PK | 58.8     			|2_295_918     | N/A |
| Postgresql fdw table  (15M Rows x  9c columns) HEAP Target | 40.1     			|3_366_583     | N/A |